# Rock Climbing Logbook (8a.nu) Analysis
## or: Does Being Tall *Really* Help? <br>
Steve Bachmeier <br>
2019-03-15

In [3]:
# Code to run for report

#----------------------------------------------------------------
# Run the following to hide the In[] and Out[] margin. 
# Doing so will not allow headings to be collapsed.
'''
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
'''

#----------------------------------------------------------------
# Run the following to import required libraries
import dill

#----------------------------------------------------------------
# Run the following to load required pickled objects

df_all_head = dill.load(open("df_all_head.pkl", "rb"))

## 1 Synopsis

## 2 Overview

### 2.1 Background

Rock climbing is a tough sport. It takes physical strength, extreme endurance in strange muscle and tendon groups that are not typically targeted (finger ligaments, forearms, etc), dancer-like body awareness and balance (although I am failing hard in this arena!), and serious mental grit. With such obstacles to overcome, then, nothing is more infuriating than when I finally - finally! - successfully complete a climb without falling only to hear such grumblings as: "Ugh, it's so easy if you're tall" or "It's not fair - you can just reach past the difficult holds!"

It's true that with a height of six feet and a +2 so-called ape index (that is, my tip-to-tip arm span is my height plus two inches, ie 6'2") I can sometimes do in one big move what a shorter guy or gal might require two or three. But conversely, and I've argued this adamently for years, there are plenty of times where the next hold is perfectly at arms length for a short person whereas for me it's relatively lower and requires me to scrunch up into all sorts of weird inefficient positions. The more I crunch, the more my butt sticks out, and the farther from the wall my center of gravity goes - it's all physics! That's not even to mention the math behind leverage (where my longer arms are not necessarily an advantage) or the fact that taller people also tend to be heavier. 

So what's the deal? Is it better for climbing to be tall or short? And while we are at it, what about other factors like weight, gender, and years of experience?

### 2.2 Data

The raw data was downloaded from: https://www.kaggle.com/dcohen21/8anu-climbing-logbook.

The data used was scraped from an online logbook, https://beta.8a.nu/, by David Cohen (https://www.kaggle.com/dcohen21). It should be noted that the user's code to scrape the website is no longer avaialable due to DMCA takedown.

Per the data description, it was collected on 2017-9-13. The data is assumed to be accurate.

### 2.2.1 8a.nu description

8a.nu is a rather popular online logbook used to track rock climbs completed. A user creates an account, searches for a specific climb he/she finished, and logs information about it such as its difficulty; whether it was on-sighted (climbed without falling on the first time), flashed (climbed without falling on the first time but after having watched someone else do it), or redpointed it (climbed it without falling after previously failing); how much the liked it based on a 0-3 star rating; and any other notes.

The website also includes details about each climb such as the consensus difficulty grade and rating, location, and type of climb ('rope' refers to taller climbs that are typically done with a harness/rope and belayer and 'boulder' refers to shorter climbs done with no rope).

Finally, user details can be optionally input including things such as gender, height, weight, the year he/she began climbing, etc.

**It is important to note that climbs on 8a.nu are only logged when successfully completed without falling.**

### 2.3 Goal

To determine which - if any - attributes give climbers a statistically significant advantage.

## 3 Data wrangling
This section outlines the data formatting completed. Refer to the Jupyter Notebook analysis.ipynb for entire analysis and code.

### 3.1 Data preparation

The downloaded dataset from came in the form of an SQlite database. Using the Python ```sqlite3``` package, all tables in the database were read into ```pandas``` dataframes and then saved out as csv files. All csv files were then read back into Python into one dictionary which was finally unpacked into separate raw data dataframes; there were in the end four:
* users (62592 rows, 22 columns)
* ascents (~4.1 million rows, 28 columns)
* climbing methods (5 rows, 4 columns)
* climbing grades (83 rows, 14 columns)

Each row in the users dataframe corresponded to a different unqique person and included such information ask user ID, name, location, gender, height, weight, etc.

The ascents dataframe is a log of all of the climbs logged on 8a.nu and includes information like the ascent ID, the user ID of that specific ascent, the method ID, date, notes, etc.

The method dataframe is a small one that includes ony five rows. It's columns consist of method ID, score, shorthand, and name. Shorthand and name are redundant (lower case vs capitalized) and are simply the types of finish a climber can get (redpoint, flash, onsight, and toprope). It's unclear what exactly score is. Note that there is not option for failed attempts (finishing but having fallen at least once in the process).

Finally, the grades dataframe lists all of the current climbing grade possibilities from very easy (grade ID of 0) to world class (grade ID of 82). The grade IDs align one-to-one with the French rating system (0 to 9c+/10a). Other columns are included (including the American Yosemite Decimal System grading system) but the fact that the French grades align completely with the grade IDs implies that grade ID is a good indicator of difficulty. Essentially, it is assumed that climbs become linearly difficult from a grade ID of 0 up to a grade ID of 82.

### 3.2 Data cleaning

With the raw data now in usable dataframes, they can be cleaned. Refer to analysis.ipynb for the code itself.

In summary:

1. Drop unnecessary columns from all four dataframes
2. Check for null/empty values in all dataframes $^{[a]}$
3. For the user dataframe:
  1. Convert the 'sex' variable to dummy variable 'is_female' $^{[b]}$
  2. For the birth years that exist, extract just the year.
  3. Drop any remaining duplicate user IDs.
4. For the ascent dataframe:
  1. Drop any duplicate ascent IDs.
  2. Convert user ID column to integers.

------------
$^{[a]}$ It should be noted that there is a fair amount of null values for the 'birth' feature in the users dataframe; 55.5% of people did not bother putting their birth date into the website. It was decided not to drop the column nor to impute the values; instead, filter as necessary.

$^{[b]}$ The 'sex' variable had three unique values. The vast majority were 0s and 1; four users had sex=255. As I highly doubt that the 8a.nu team has implemented non-binary gender assignment functionality into the website, those four users were simply dropped from the dataset. 

As for the 0s and 1s, it was found that 0s were on average heavier and taller than 1s. Further, there were significantly more 0s than 1s. As most climbers are male and males tend to be taller and heavier than females, the 'sex' category was renamed as 'is_female'. 

------------

### 3.3 Data merge and new feature creation

With cleaned datasets, they can now be merged into one large working dataframe. This is completed by merging the user dataframe onto the ascent dataframe using the user ID primary keys. The columns of the new working dataframe are then rearranged and two new features added: 'age' ('year' - 'birth_year') and 'years_climbed' ('year' - 'started').

With the new 'age' and 'years_climbed' features created, the 'year', 'birth_year', and 'started' columns are no longer of interest and are dropped.

Finally, to clarify some of the column names (and remain consistent), the following labels are renamed:
* 'user_id' to 'id_user'
* 'method_id' to 'id_method'
* 'grade_id' to 'id_grade'
* 'climb_type' to 'is_bouldering' $^{[a]}$

-------------

Notes

$^{[a]}$ The 'climb_type' variable is binary and consists of 0s and 1s. By comparing the climbs with the most 0s and the most 1s (ie the most popular climbs for each 'climb_type') to their descriptions on another popular online log (www.mountainproject.com), it became clear that 'climb_type' = 0 refers to rope climbs and 'climb_type' = 1 refers to bouldering problems. As such, the variable was renamed to 'is_bouldering' to reduce any future confusion.

--------------

### 3.4 Filter data

Recall that there are four different method types (redpoint, flash, onsight, and toprope). For the purposes of this analysis, toprope ascents are not considered successful $^{[a]}$ and so all ascents with an 'id_method' = 4 are dropped from the working dataframe. Then the 'id_method' column is dropped.

At this point the working dataframe consists of ~4 million rows and 9 columns.

--------------

Notes:

$^{[a]}$ Toproping is when a person climbs attached to the rop which goes up to an anchor at the top of the climb and then back down to the belayer. 

--------------

#### 3.4.1 A note about NULL/bogus values

It should be noted here that, despite having cleaned up the data already, NULL and bogus values have been left intact on purpose. Specifically, the date data ('years_climbing' and 'age') is full of NULL values as well as values that simply don't make sense (negatives, 0s, and impossibly large values). Likewise, the weight data includes some 0s and the height data includes values ranging from 0cm tall to 255cm tall. 

All of this data is purposefully left in because to remove them all would drastically reduce the dataset even for variables that are otherwise fine. For example, over 21% of the observations have NULL values for 'age' (as a result of either the original 'year' or 'birth_year' being NULL). However, just because users chose not to input year data does not mean that the rest of their data is bad! As such, the entirety of the dataset is kept intact and the following filters will be applied as necessary:

* 'height': 120-240 cm 
* 'weight': not 0 (equivalent to 40-100 kg)
* 'age': 10-50
* 'years_climbing': 0-40

**Note that these filters are somewhat arbitrary!** There certainly may be climbers in the dataset shorter than 120 cm, taller than 240 cm, with more than 40 years of climbing experience, or out outside the age range of 10-50. However, it is assumed that most users that have these outlier vales are (a) either just that, outliers or (b) simply input incorrect data. 

### 3.5 Reduce

To accurately gage a climber's skill level, it is important not to consider every single grade ever climbed but instead to extract each person's personal best. Much like a world-class sprinter does not always run to beat a record, climbers very often - mostly, really - climb at grades that are below their historical high. As such, the total working dataframe is reduced to only include each user's *oldest* maximum 'id_grade' for both rope climbs and boulder problems.

With the 'id_ascent' column no longer needed for such sorting and grouping, it is dropped.

The head of the working dataframe is shown below.

In [4]:
df_all_head

,id_user,is_female,height,weight,is_bouldering,id_grade,age,years_climbing
0,1,0,177,73,0,62,25.0,5.0
1,1,0,177,73,1,51,26.0,6.0
2,2,0,0,0,0,49,NaN,1.0
3,2,0,0,0,1,46,NaN,1.0
4,3,0,180,78,0,62,26.0,4.0
5,3,0,180,78,1,55,28.0,6.0


### 3.6 Separate data

Recognizing that rope climbing and bouldering are quite different despite both being considered "rock climbing," it was decided to separate the dataset based on 'is_bouldering'. Similarly, the datasets are separated on 'is_female' as well to get different results based on gender. There are thus five different dataframes to be used depending on the analysis at hand:

* df_all (49,598 rows, 8 columns)
* df_bouldering_female (2,396 rows, 6 columns)
* df_bouldering_male (16,817 rows, 6 columns)
* df_rope_female (4,592 rows, 6 columns)
* df_rope_male (25,793 rows, 6 columns)

Note that the four data subsets contain six columns instead of eight like ```df_all```; this is because 'is_female' and 'is_bouldering' are not useful and are dropped.

## 4 Exploratory data analysis
It is always a good idea to do at least a bit of exploratory data analysis before diving too deep into an analysis; it can shed light on trends, potential problems, etc.

#### 3.4.1 Pairplot

A pairplot of all of the non-dummy variables is shown below; successful projects are green while failed are blue. There does not seem to be good separation of *launch_state* values with the exception perhaps of *goal*: it does appear as if projects with large goals have few successes.

<img src="images/pair_plot.jpeg" style="width: 100%;"/>

#### 3.4.2 *funding_days* vs *goal*

Below shows *funding_days* vs *goal*; the left plot shows all data points while the right is zoomed in on the *goal* (x-) axis to $0-$250,000. We can see that while there is no significant separation at these lower goal levels, there might be a slight benefit to having longer *funding_days*. Successful launches seem loosely clustered around *funding_days* = [0,60] and *goal* < $100k.

<table><tr><td><img src='images/funding_days_vs_goal.jpeg'></td><td><img src='images/funding_days_vs_goal_2.jpeg'></td></tr></table>

#### 3.4.3 *staff_pick*

As mentioned above in the [Variable - outcome correlation](#3.3.3) section, *staff_pick* has a relatively high correlation with the outcome of 0.25. Indeed, the images below show that projects chosen as a staff pick (*staff_pick* = 1) rarely fail to become successfully funded.

<table><tr><td><img src='images/staff_pick_vs_launch_state.jpeg'></td><td><img src='images/staff_pick_vs_launch_state_2.jpeg'></td></tr></table>

The bar plot below shows that the average *launch_state* for *staff_pick* of 0 and 1 is 0.523 and 0.889, respectively. Since *launch_state* is binary with 0 for failures and 1 for successes, these means also represent successful launch percentages, ie 52.3% of projects that are not chosen as a staff pick succeed while 88.9% of projects that are chosen succeed.

<img src="images/average_launch_state_vs_staff_pick.jpeg" style="height: 400px;"/>

It should also be noted that 13.5% of the projects were chosen as staff picks. From https://www.kickstarter.com/blog/how-to-get-featured-on-kickstarter, it appears as if projects are featured when they catch the eye of the Kickstarter staff via creativity, a nice and visually appealing site, etc. ie, they are NOT just picked due to them being funded well.

Clearly, being chosen as a staff pick is correlated with funding success. What is unclear, however, is whether getting chosen actually helps in success or if projects that were already going to be successful are chosen.

#### 3.4.4 Categories

There are 15 main categories that a project can fall under (with hundreds of sub-categories). A frequency plot of these 15 categories is shown below. The two most common categories are 'film & video' and 'music'.

<img src="images/category_freq_plot.jpeg" style="height: 400px;"/>

Note that the above plot says nothing of the success of these categories. For that, I've plotted the average *launch_state* value (which, again, can be interpreted as the percentage of successes) as a function of category, below. The three most successful categories are 'comics', 'dance', and 'publishing' while the least successful are 'journalism', 'technology', and 'food'.

<img src="images/launch_state_vs_category_barplot.jpeg" style="height: 400px;"/>

To better show that some categories have better chances of success than other, I've plotted a heatmap and a clustermap, below. Successes (*launch_state* = 1) are white and failures (*luanch_state* = 0) are black. I used 0.5 for empty cells so as not to bias towards success or failure since a project can only be one category and so most values will indeed be empty. As such, in both figures, a mostly-white column can be interpreted as a highly successful category while a mostly-black column can be interpreted as not very successful. It can be clearly seen that, as indicated above, 'journalism' and 'technology' categories have a relatively high failure rate.

<img src="images/category_heatmap.jpeg" style="height: 400px;"/>

<img src="images/category_clustermap.jpeg" style="height: 400px;"/>

### 3.5 Model exploration

Refer to appendix [A1.4 Model exploration](#A1.4) for code.

To start creating the machine learning models, I separated the working dataset into train and test sets. Note that using 25% of the working set as the test size ensures that 20% of the entire raw set is reserved for the test set (this is not exact since we did some cleaning after creating the validation set, but it's close). Also note that we dropped the outcome *launch_state* variable as well as several information-only variables from the dataset to be used for machine learning variables, X.

```
info_variables = ['id','launched_at','category','country', 'pledged_ratio', 'backers_count']

X = df.drop(columns=info_variables).drop(columns='launch_state')
y = df['launch_state']

#-----------------------------------------
# TRAIN/TEST SPLIT

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=101)
```

Next, the training and test datasets were scaled.

```
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
```

With the data scaled, I built and analyzed all of the models of interest by
1. loading the relevant classifier.
2. fitting the training data and outcome vector to the classifier.
3. predicting the test outcomes using the trained classifier on the test data.
4. printing the confusion matrix and the classification report comparing the known test outcomes to the predicted outcomes.
5. calculating the one-run accuracy as the sum of the confusion matrix diagonal divided by the sum of the entire confusion matrix.
6. completing a 10-fold cross validation for models that did not take too long to run.
7. printing the 10-fold cross validation accuracies, mean accuracy, and accuracy standard deviation.

Again, all code can be found in [A1.4 Model exploration](#A1.4).

The results of these initial exploratory runs are shown in the table below.

In [7]:
df_results[0:7]

,model,time_fit,time_predict,time_10_fold_CV,accuracy,acc_10_fold
0,Naive Bayes,0.101767,0.036270,1.53579,0.623551,0.624725
1,Logistic Regression,1.822522,0.009085,17.0696,0.694333,0.689984
2,K Nearest Neighbors,7.340739,101.319590,344.234,0.682739,0.675807
3,"SVM, Linear",954.375944,90.726687,None,0.677168,None
4,"SVM, RBF",1091.777649,120.427602,None,0.682908,None
5,Decision Tree,0.437127,0.014542,4.82765,0.664357,0.662544
6,Random Forest (10-fold),1.016743,0.104582,10.463,0.683248,0.680784


As can be seen above, all of the models seem to have similar accuracies (between ~62% and ~69%) while their run times vary greatly. 

A quick look into variable reduction via principal component analysis (PCA) was then completed. The code for this is found in Appendix [A1.4.9 Principal component analysis](#A1.4.9). Fitting the training data to a PCA object with only the top two principal components results in the following scatter plot. As expected (since there was not great separation when all of the variables were included), it does not seem like reducing the variables to their principal components make sense.

<img src="images/pca_2components.jpeg" style="height: 400px;"/>

For completeness, the PCA object with two principal components was fit to the Naive Bayes algorithm. It ran very fast (about 5x faster than the original Naive Bayes model) and head nearly the same accuracy (61% instead of 62%). This would be a great approach in some cases to speed things up while maintaining similar accuracy! The results table below now includes this new PCA result.

In [10]:
df_results[0:8]

,model,time_fit,time_predict,time_10_fold_CV,accuracy,acc_10_fold
0,Naive Bayes,0.101767,0.036270,1.53579,0.623551,0.624725
1,Logistic Regression,1.822522,0.009085,17.0696,0.694333,0.689984
2,K Nearest Neighbors,7.340739,101.319590,344.234,0.682739,0.675807
3,"SVM, Linear",954.375944,90.726687,None,0.677168,None
4,"SVM, RBF",1091.777649,120.427602,None,0.682908,None
5,Decision Tree,0.437127,0.014542,4.82765,0.664357,0.662544
6,Random Forest (10-fold),1.016743,0.104582,10.463,0.683248,0.680784
7,"PCA (n=2), Naive Bayes",0.024343,0.003595,0.284533,0.609355,0.608927


### 3.6 Model tuning and selection

Refer to appendix [A1.5 Model tuning and selection](#A1.5) for code.

At this point it was time to choose some models to tune and make a final selection. This was completed for three models using sklearn's *GridSearchCV* method to hone in on the parameters that optimize the results. These optimal parameters are outlined below:

* Random forest:
    * *n_estimators* = 100
    * *criterion* = "gini"
    * *max_features* = "sqrt"
    * *min_samples_leaf* = 25

* Logistic regression:
    * *C* = 10
    * *penalty* = "l1"

* K nearest neighbors:
    * *metric* = "minkowski"
    * *n_neighbors* = 23
    * *p* = 2

We also plotted validation curves for the numeric parameters to double-check for over-fitting. These curves are shown below (**Note: The image of the KNN validation curve was unortunately not saved before the entire session was pickled and dumped, ie I do not have that particular plot**). Note that they agree with the above outline; the models are made complicated enough to minimize variance but not so complicated that bias becomes unnecessarily large. Interestingly, the training and cross-validation curves do not start deviating from each other (high bias) when the random forest mode's *n_neighbors* is high which is what I would expect from an over-fitted model.

<table><tr><td><img src='images/validation_curves_random_forest.jpeg'></td><td><img src='images/validation_curves_logistic_regression.jpeg'></td></tr></table>

The three models with optimized parameters were then run and results appended to the results table as shown below.

In [16]:
df_results

,model,time_fit,time_predict,time_10_fold_CV,accuracy,acc_10_fold
0,Naive Bayes,0.101767,0.036270,1.53579,0.623551,0.624725
1,Logistic Regression,1.822522,0.009085,17.0696,0.694333,0.689984
2,K Nearest Neighbors,7.340739,101.319590,344.234,0.682739,0.675807
3,"SVM, Linear",954.375944,90.726687,None,0.677168,None
4,"SVM, RBF",1091.777649,120.427602,None,0.682908,None
5,Decision Tree,0.437127,0.014542,4.82765,0.664357,0.662544
6,Random Forest (10-fold),1.016743,0.104582,10.463,0.683248,0.680784
7,"PCA (n=2), Naive Bayes",0.024343,0.003595,0.284533,0.609355,0.608927
8,Random Forest (Optimized),5.856856,0.470576,66.9746,0.718257,0.716067
9,Logistic Regression (Optimized),6.454474,0.003956,68.8582,0.695662,0.691059


As shown above, the prediction accuracy increases slightly for all three models when using optimized parameters. An optimized random forest model provides the best accuracy at ~72% while at the same time remaining efficient and fast to run. This is the chosen model for future use.

## 4 Prediction / final validation

Refer to appendix [A2 Code - new data prediction](#A2) for relevant code.

With one specific model chosen to launch, it is time for a final validation. In this case, this validation also serves as a proving grounds for the final product. Recall that thus far we have not touched 20% of the initial raw data - this untouched set was set aside as a validation set. Keeping a completely separated validation set is useful because it ensures that the model in no way relied on it. This is in contrast to the training set which was used extensively to fit and train the model. Even the test set was used indirectly to baseline the model when doing accuracy comparisons. Only the validation set has been 100% unused.

The goal here is to then create a script that takes in raw Kickstarter project data (again, in this case that raw data is the validation set previously set aside), cleans it up, and uses the chosen machine learning algorithm to predict which projects will be successfully funded or not. Since we do have the real-life outcomes of this set, we can also calculate our final product accuracy.

Aside from some extra code that asks the user for input, runs various handling exceptions, and deals with whether or not the input file is a truly raw JSON file or whether it is an extracted dataframe from the JSON file (as is the case here with the validation set), the product script is quite simple. It 

1. takes in the raw data.
2. cleans the data.
3. extracts the relevant machine learning columns as well as the outcome column (if one exists).
4. applies the prediction algorithm.
5. writes out the prediction vector to a CSV file.
6. evaluates the accuracy of the prediction vector to the known values, if applicable. Specifically, it prints the confusion matrix, classification report, and calculates the accuracy as the sum of the confusion matrix diagonal divided by the sum of the entire confusion matrix.

### 4.1 Instructions

Running the script is quite simple. 
1. Ensure that Python is properly installed.
2. Ensure that the following files are located in the working directly:
	* classifier_rf_opt.pkl
	* f_cleanData.py
	* f_dataImport.py
	* f_predict.py
	* predict.py
	* sc_X.pkl
3. Create a folder named 'data' in the working directory.
4. Download or create the Kickstarter data to run the prediction model on.
	* The data must be in JSON format like from https://webrobots.io/kickstarter-datasets/.
	* Alternatively, the data can a comma-separated value dataframe from previously-run analyses.
5. Save the raw data in the 'data' folder.
6. Open a command prompt.
7. Type 'python predict.py'
8. Follow the prompts.

A screen shot of this process is shown below.

<img src="images/validation_cmd.jpg" style="width: 100%;"/>

The above image also shows the confusion matrix, classification report, and calculated accuracy. As expected, **the model performs admirably with a 70% accuracy and managed to complete the analysis in only 1.49 seconds**. Further, a copy of the prediction vector was saved in the home folder.

## 5 Next steps

Some recommendations to improve this analysis include:

* Explore and account for any outliers that exist.
* Use datetime information instead of simply extracting the years to get more accurate age and years_climbing data.
* Explore the *rate* at which each user progresses.
* User more data! Users of 8a.nu tend to be rather strong climbers and so this analysis leans very much towards those who have made rock climbing a significant part of their life. It therefore may not be appropriate to draw conclusons about more casual rock climbers.